In [32]:
!pip install POT

In [33]:
import pandas as pd
import numpy as np
import gdown
import gensim.downloader as api
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import jaccard, dice, hamming
from scipy.stats import pearsonr, entropy
from sklearn.cluster import KMeans
from gensim.corpora import Dictionary
from gensim.similarities import WmdSimilarity
from gensim.models import KeyedVectors
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url = "https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i"
output = "dataset.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i
To: /content/dataset.csv
100%|██████████| 93.0M/93.0M [00:00<00:00, 175MB/s]


'dataset.csv'

In [34]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
glove_vectors = api.load("glove-wiki-gigaword-300")

In [36]:
input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_text = input_text.lower()
input_text = re.sub(r'[^\w\s]', '', input_text)
tokens = word_tokenize(input_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(word) for word in tokens]
input_text = ' '.join(tokens)

In [37]:
def get_average_glove(tokens, model, num_features):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)
    return np.mean([model[word] for word in valid_words], axis=0)

In [38]:
dataset = pd.read_csv(output)

In [39]:
corpus_tokens = [word_tokenize(re.sub(r'[^\w\s]', '', doc.lower())) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_tokens = [[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] for tokens in corpus_tokens]
num_features = 300
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, num_features) for tokens in corpus_tokens])

In [40]:
input_tokens = word_tokenize(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, num_features)

In [41]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [42]:
euclidean_distance = euclidean_distances([input_embedding], corpus_embeddings).flatten()
dataset["euclidean_distance"] = euclidean_distance

In [43]:
manhattan_distance = manhattan_distances([input_embedding], corpus_embeddings).flatten()
dataset["manhattan_distance"] = manhattan_distance

In [44]:
def jaccard_similarity(s1, s2):
    s1 = set(s1.split())
    s2 = set(s2.split())
    return len(s1 & s2) / len(s1 | s2)
jaccard_similarities = np.array([jaccard_similarity(' '.join(input_tokens), ' '.join(tokens)) for tokens in corpus_tokens])
dataset["jaccard_similarity"] = jaccard_similarities

In [45]:
def dice_coefficient(s1, s2):
    s1 = set(s1.split())
    s2 = set(s2.split())
    return 2 * len(s1 & s2) / (len(s1) + len(s2))
dice_coefficients = np.array([dice_coefficient(' '.join(input_tokens), ' '.join(tokens)) for tokens in corpus_tokens])
dataset["dice_coefficient"] = dice_coefficients

In [46]:
def pearson_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0
    return pearsonr(x, y)[0]
pearson_correlations = np.array([pearson_corr(input_embedding, emb) for emb in corpus_embeddings])
dataset["pearson_correlation"] = pearson_correlations

In [47]:
def wmd_distance(s1, s2, model):
    s1_words = s1.split()
    s2_words = s2.split()
    s1_words = [word for word in s1_words if word in model]
    s2_words = [word for word in s2_words if word in model]
    if not s1_words or not s2_words:
        return float('inf')
    return model.wmdistance(s1_words, s2_words)
wmd_distances = np.array([wmd_distance(' '.join(input_tokens), ' '.join(tokens), glove_vectors) for tokens in corpus_tokens])
dataset["wmd_distance"] = wmd_distances

In [48]:
def bhattacharyya_distance(p, q):
    p = np.sqrt(np.abs(p))
    q = np.sqrt(np.abs(q))
    return -np.log(np.sum(p * q) + 1e-10)
def bhattacharyya_distance_from_vector(x, y):
    p = x / (np.sum(x) + 1e-10)
    q = y / (np.sum(y) + 1e-10)
    return bhattacharyya_distance(p, q)
bhattacharyya_distances = np.array([bhattacharyya_distance_from_vector(input_embedding, emb) for emb in corpus_embeddings])
dataset["bhattacharyya_distance"] = bhattacharyya_distances

In [49]:
def kl_divergence(p, q):
    epsilon = 1e-10
    p = p + epsilon
    q = q + epsilon
    p = p / np.sum(p)
    q = q / np.sum(q)
    return entropy(p, q)
kl_divergences = np.array([kl_divergence(input_embedding, emb) for emb in corpus_embeddings])
dataset["kl_divergence"] = kl_divergences

# Hasil

In [50]:
pd.set_option("display.max_colwidth",None)

In [51]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(20)
top_similar[["skill","job title","cosine_similarity"]]

,skill,job title,cosine_similarity
13111,Information Technology,data science artificial intelligence machine learning internship,0.935107
9203,Other,senior bioinformatics research scientist bioinformatic research scientist,0.931558
22397,Research,group summer research program intern advanced concept technology,0.930636
19613,Analyst,data scientist,0.928304
29493,Education,researcher doctor of philosophy student,0.927795
4277,Science,senior data scientist,0.927549
4486,Science,senior data scientist,0.927549
19252,Information Technology,data scientist,0.927438
1784,Project Management,senior manager,0.927376
19401,Engineering,research development civil engineering,0.927259


In [52]:
top_similar['cosine_similarity']

13111    0.935107
9203     0.931558
22397    0.930636
19613    0.928304
29493    0.927795
4277     0.927549
4486     0.927549
19252    0.927438
1784     0.927376
19401    0.927259
1142     0.924037
5171     0.923820
6180     0.923580
23306    0.923451
7319     0.923152
28867    0.922734
23588    0.922225
30445    0.921715
4711     0.921482
30759    0.921435
Name: cosine_similarity, dtype: float64

In [53]:
top_similar = dataset.sort_values(by='euclidean_distance', ascending=True).head(10)
top_similar

id_pekerjaan  id_perusahaan                   skill  \
13111    3701314004       71049830  Information Technology   
22397    3757722520           1504                Research   
9203     3699053978       15223686                   Other   
19613    3757709060          23099                 Analyst   
19401    3749350135        9247671             Engineering   
4277     3693581630           2276                 Science   
4486     3693584453           2276                 Science   
19252    3757450438         871133  Information Technology   
1784     3693049684       18581793      Project Management   
1142     3693046569       10577525      Project Management   

                                                                       job title  \
13111           data science artificial intelligence machine learning internship   
22397           group summer research program intern advanced concept technology   
9203   senior bioinformatics research scientist bioinformatic research scientist   
19613                                                             data scientist   
19401                                     research development civil engineering   
4277                                                       senior data scientist   
4486                                                       senior data scientist   
19252                                                             data scientist   
1784                                                              senior manager   
1142                                                       data scientist remote   

      jenis_pekerjaan   registration_type category sponsor  \
13111      Internship  ComplexOnsiteApply        sponsored   
22397       Full-time        OffsiteApply    not sponsored   
9203        Full-time        OffsiteApply    not sponsored   
19613        Contract  ComplexOnsiteApply    not sponsored   
19401       Full-time  ComplexOnsiteApply    not sponsored   
4277        Full-time        OffsiteApply        sponsored   
4486        Full-time        OffsiteApply    not sponsored   
19252       Full-time  ComplexOnsiteApply    not sponsored   
1784        Full-time  ComplexOnsiteApply        sponsored   
1142        Full-time  ComplexOnsiteApply    not sponsored   

                         city          state         county  \
13111  san francisco bay area     california  united states   
22397               lexington  massachusetts  united states   
9203                  memphis      tennessee  united states   
19613                 houston          texas  united states   
19401               vicksburg    mississippi  united states   
4277      south san francisco     california  united states   
4486      south san francisco     california  united states   
19252                new york       new york  united states   
1784   san francisco bay area     california  united states   
1142                 new york       new york  united states   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [54]:
top_similar = dataset.sort_values(by='manhattan_distance', ascending=True).head(10)
top_similar["manhattan_distance"]

13111    13.712642
19613    14.012262
4277     14.070134
4486     14.070134
22397    14.181440
19401    14.265179
9203     14.292675
1784     14.314301
23306    14.598262
19252    14.617533
Name: manhattan_distance, dtype: float64

In [55]:
top_similar = dataset.sort_values(by='jaccard_similarity', ascending=False).head(20)
top_similar[["skill","job title","jaccard_similarity"]]

,skill,job title,jaccard_similarity
22536,Engineering,senior software scientist medical imaging,0.152174
9283,Information Technology,tableau architect,0.144578
12904,Engineering,financial planning consultant,0.137931
24110,Engineering,data scientist,0.134021
17502,Engineering,tech lead data analytics,0.132184
3405,Product Management,product manager,0.131455
2243,Information Technology,business intelligence lead,0.128571
24805,Research,business analyst,0.128571
2312,Information Technology,natural language processing engineer java,0.126506
13387,Engineering,senior director technology,0.126087


In [56]:
top_similar = dataset.sort_values(by='dice_coefficient', ascending=False).head(10)
top_similar["dice_coefficient"]

22536    0.264151
9283     0.252632
12904    0.242424
24110    0.236364
17502    0.233503
3405     0.232365
2243     0.227848
24805    0.227848
2312     0.224599
13387    0.223938
Name: dice_coefficient, dtype: float64

In [57]:
top_similar = dataset.sort_values(by='pearson_correlation', ascending=False).head(10)
top_similar["pearson_correlation"]

13111    0.935447
9203     0.932351
22397    0.930595
19613    0.928910
19401    0.927988
29493    0.927784
1784     0.927726
19252    0.927703
4486     0.927613
4277     0.927613
Name: pearson_correlation, dtype: float64

In [58]:
filtered_dataset = dataset[dataset["wmd_distance"] != float('inf')]
top_similar = filtered_dataset.sort_values(by='wmd_distance', ascending=True).head(10)
top_similar["wmd_distance"]

19252    0.851335
18373    0.858288
1885     0.869854
12904    0.873789
17502    0.876112
9283     0.876529
28867    0.877625
1784     0.881151
30759    0.886625
30445    0.889444
Name: wmd_distance, dtype: float64

In [59]:
top_similar = dataset.sort_values(by='bhattacharyya_distance', ascending=True).head(10)
top_similar["bhattacharyya_distance"]

11720   -5.856735
15245   -5.764600
6985    -5.466965
24799   -5.385907
490     -5.354042
20477   -5.311746
10681   -5.286391
24857   -5.234039
11325   -5.226479
31511   -5.180870
Name: bhattacharyya_distance, dtype: float64

In [60]:
filtered_dataset = dataset[dataset["kl_divergence"] != float('inf')]
top_similar = filtered_dataset.sort_values(by='kl_divergence', ascending=True).head(10)
top_similar["kl_divergence"]

Series([], Name: kl_divergence, dtype: float64)